### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/"

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [67]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 로그인 정보: EP 정보로 입력해주세요
    'LOGIN_ID': 'ws.jang', # "cism-dev"
    'LOGIN_PW': '', # "cism-dev@com"
    
    # workspace 이름 
    'WORKSPACE_NAME': "magna-ws", # "cism-ws"
    
    # 시스템 URI
    'URI': "https://aic-web-kic.aidxlge.com/", 
    
    # ECR에 올라갈 컨테이너 URI TAG 
    'ECR_TAG': 'latest', 
    
    # scripts/creating_ai_solution/image/ 밑에 UI에 표시될 아이콘 이미지 파일 (ex. icon.png) 를 업로드 해주세요. 
    # 이후 해당 아이콘 파일 명을 아래에 기록 해주세요.
    'ICON_FILE': 'icon.png'
}
#----------------------------------------#

> 시스템 로그인

In [68]:
# 시각화 용 모듈 install 
!pip install tabulate

In [69]:
# Generate RegisterUtils instance & login
from register_utils import RegisterUtils
registerer = RegisterUtils(user_input)

# AI Conductor Login
registerer.login()

URI_SCOPE:  magna-ws

>> Success getting cookie from AI Conductor:
 {'access-token': 'lZfzOB7u03bwJO-Nhd97FCaIHyBkV6dFv6tDBNjGC9U'}

>> Success Login: {'result': 'OK'}


> AI Solution 이름 설정 (사용자 입력)

In [70]:
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

# soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
user_solution_name = input("- Enter the name you want to create: ")

# 기존에 존재하는 solution name과 겹치지 않는지 체크 
registerer.check_solution_name(user_solution_name)


- Enter the name you want to create:  magna-231208-1643


[Success] Allowed name: << magna-231208-1643 >>


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
&#x1F600; 이름이 등록되면 본 jupyter 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
변경이 필요할 경우 <b> STEP-2-1 </b> 부터 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [71]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# check workspace (ECR, S3정보 셋팅까지 진행)
registerer.check_workspace() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)

[{'id': '1b61dae3-e8f7-4b5f-9629-da0aedbb4a08', 'name': 'cism-ws', 'namespace': 'aic-ns-cism-ws', 'kubeflow_user': 'aic-user-cism-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-cism-dev-aia', 'private': 's3-an2-cism-dev-cism'}, 'ecr_base_path': {'public': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/public/', 'private': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/cism/'}, 'execution_specs': [{'name': 'high', 'label': 'cism-ws-high', 'vcpu': 8, 'ram_gb': 32, 'gpu': 0}, {'name': 'standard', 'label': 'cism-ws-standard', 'vcpu': 2, 'ram_gb': 8, 'gpu': 0}], 'is_deleted': 0}, {'id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'name': 'magna-ws', 'namespace': 'aic-ns-magna-ws', 'kubeflow_user': 'aic-user-magna-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-cism-dev-aia', 'private': 's3-an2-cism-dev-magna'}, 'ecr_base_path': {'public': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai

----

#### **STEP-3**. Train 용 Sample data, train artifacts, model, icon 등록

In [72]:
# data 등록 
# s3 접근확인
registerer.s3_access_check('data')

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()


[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading data into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/input/train/

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/data/train/20230117_04_26_48_LTCO75645678010008_ETC2(0)_OK.csv

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/data/train/20230117_02_37_38_06Y8004090400000XP2405031012V5545_ETC2(5)_OTHERS(0).csv

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/data/train/20230120_05_40_05_06Y8004090400000XP2405031012V5545_ETC1(1)_OK.csv

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/data/train/20230904_15_38_57_LR23180A0031_ETC1(2)_OK.csv

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/data/train/20230904_15_40_00_LR23180A0031

In [73]:
# artifacts 등록 
# s3 접근확인
registerer.s3_access_check('artifacts')

# s3 model, artifacts 업로드 
# 이전에 이미 해당 s3 경로에 존재하던 object는 지워집니다.
registerer.s3_upload_artifacts()


[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << train artifacts >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/artifacts/train_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: train


In [74]:
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
registerer.s3_upload_icon()

Success uploading into S3 path: s3-an2-cism-dev-magna/icons/magna-231208-1643/icon.png


----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. 

In [75]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/main.py "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/ " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/src "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/src " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/config "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/config " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/assets "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/assets " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/alolib "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/alolib " 

 Success ALO directory setting.


> DOCKERFILE을 셋팅합니다. 

In [76]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: train


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [77]:
## ECR 등록
## docker login 실행 
registerer.set_aws_ecr(docker=True)

p1:  ['aws', 'ecr', 'get-login-password', '--region', 'ap-northeast-2']
[INFO] target AWS ECR url: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com
p2:  ['sudo', 'buildah', 'login', '--username', 'AWS', '--password-stdin', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com']
[INFO] AWS ECR | docker login result: 
 Login Succeeded!

[INFO] Target AWS ECR repository: 
ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:042969618971:repository/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643",
        "registryId": "042969618971",
        "repositoryName": "ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643",
        "repositoryUri": "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643",
      

> Docker Build

In [78]:
# docker build 
registerer.build_docker()

STEP 1/12: FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
STEP 2/12: RUN apt-get update
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8062 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [260 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [17.7 kB]
Fetched 8548 kB in 1s (8001 kB/s)
Reading package lists...
STEP 3/12: RUN apt-get install -y apt-utils
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  apt-utils
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 439 kB of archives.
After this operation, 1192 kB of additional disk space will be used.
Get:1 http://deb.debian.org

debconf: delaying package configuration, since apt-utils is not installed


Fetched 439 kB in 0s (12.7 MB/s)
Selecting previously unselected package apt-utils.
(Reading database ... 7034 files and directories currently installed.)
Preparing to unpack .../apt-utils_2.2.4_amd64.deb ...
Unpacking apt-utils (2.2.4) ...
Setting up apt-utils (2.2.4) ...
STEP 4/12: RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20210119).
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu bzip2 cpp cpp-10 dpkg-dev
  g++ g++-10 gcc-10 git-man libasan6 libatomic1 libbinutils libbrotli1
  libc-dev-bin libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0 libctf0
  libcurl3-gnutls libdpkg-perl liberror-perl libgcc-10-dev libgdbm-compat4
  libgomp1 libisl23 libitm1 libldap-2.4-2 liblsan0 libmp

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.32.1 /usr/local/share/perl/5.32.1 /usr/lib/x86_64-linux-gnu/perl5/5.32 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.32 /usr/share/perl/5.32 /usr/local/lib/site_perl) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7, <> line 56.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 78.9 MB in 1s (131 MB/s)
Selecting previously unselected package perl-modules-5.32.
(Reading database ... 7115 files and directories currently installed.)
Preparing to unpack .../00-perl-modules-5.32_5.32.1-4+deb11u2_all.deb ...
Unpacking perl-modules-5.32 (5.32.1-4+deb11u2) ...
Selecting previously unselected package libgdbm-compat4:amd64.
Preparing to unpack .../01-libgdbm-compat4_1.19-2_amd64.deb ...
Unpacking libgdbm-compat4:amd64 (1.19-2) ...
Selecting previously unselected package libperl5.32:amd64.
Preparing to unpack .../02-libperl5.32_5.32.1-4+deb11u2_amd64.deb ...
Unpacking libperl5.32:amd64 (5.32.1-4+deb11u2) ...
Selecting previously unselected package perl.
Preparing to unpack .../03-perl_5.32.1-4+deb11u2_amd64.deb ...
Unpacking perl (5.32.1-4+deb11u2) ...
Selecting previously unselected package bzip2.
Preparing to unpack .../04-bzip2_1.0.8-4_amd64.deb ...
Unpacking bzip2 (1.0.8-4) ...
Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../05

Getting image source signatures
Copying blob sha256:1b6fd3ad4ce602924fffb84437331a255e2a9463531a1bd92a15e9e3c4d11523
Copying blob sha256:12d601beac88c4f6011b0d711827f0a08383e99e300eebb128ee17ccbb935e10
Copying blob sha256:277bd6c0df20bd4b73a9dcb29dfbe5019209893d5a8078f3189fc801384555fe
Copying blob sha256:6b3d0b913afc500b93afb9ffbff3c144b25bc17d5780bccd3979ecb29ee1e472
Copying blob sha256:2f33dd8c934e3dbab052f70b9a53cd46dbc810a9fce8cd68609bff634d8a2f00
Copying blob sha256:da2879eba471adf55f259342e4b4aa66f4eadf009b9fa8caaa197a85d9a94f83
Copying config sha256:31a25762c86cd7a54c0a504dd52cd68177a7be2a4a46758df91d0622f55ef669
Writing manifest to image destination
Storing signatures


--> 31a25762c86
Successfully tagged 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643:latest
31a25762c86cd7a54c0a504dd52cd68177a7be2a4a46758df91d0622f55ef669


> ECR에 docker image를 push 합니다. 

In [79]:
registerer.docker_push()

debug:  ['sudo', 'buildah', 'push', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643:latest']


Getting image source signatures
Copying blob sha256:6b3d0b913afc500b93afb9ffbff3c144b25bc17d5780bccd3979ecb29ee1e472
Copying blob sha256:2f33dd8c934e3dbab052f70b9a53cd46dbc810a9fce8cd68609bff634d8a2f00
Copying blob sha256:12d601beac88c4f6011b0d711827f0a08383e99e300eebb128ee17ccbb935e10
Copying blob sha256:da2879eba471adf55f259342e4b4aa66f4eadf009b9fa8caaa197a85d9a94f83
Copying blob sha256:277bd6c0df20bd4b73a9dcb29dfbe5019209893d5a8078f3189fc801384555fe
Copying blob sha256:1b6fd3ad4ce602924fffb84437331a255e2a9463531a1bd92a15e9e3c4d11523
Copying config sha256:31a25762c86cd7a54c0a504dd52cd68177a7be2a4a46758df91d0622f55ef669
Writing manifest to image destination
Storing signatures


Removed login credentials for all registries


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [80]:
registerer.set_container_uri() 

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643


----

#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [81]:
## user parameter 입력
registerer.set_candidate_parameters() # todo: selected params?
## artifact 저장 경로 지정
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << candidate_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/artifacts/


----

#### **STEP-6**. Training 에 사용될 Cloud resource 선택

> 컴퓨팅 자원을 선택합니다. (default: standard)

In [82]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

train_resource = 'standard'

#----------------------------------------#

# 클라우드 컴퓨팅 리소스 선택 
registerer.set_resource(train_resource)


[train] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-7**. Inference 용 Sample Data 등록

In [83]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check('data')
registerer.s3_upload_data()


<< solution_metadata.yaml >> updated. - appended pipeline: inference

[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << data >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/input/inference/

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/inference/data/test/20231006_16_57_06_LR23235A0040_ETC1(2)_OK.csv

Success updating solution_metadata.yaml - << dataset_uri >> info. / pipeline: inference


In [84]:
registerer.s3_access_check('artifacts')
registerer.s3_upload_artifacts()


[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << inference artifacts >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/inference/artifacts/inference_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: inference

[INFO] Start uploading << model >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/.temp_model_dir/

Success uploading into S3: 
s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/artifacts/model.tar.gz

Success updating solution_metadata.yaml - << model_uri >> info. / pipeline: inference


----

#### **STEP-8**. Inference 용 Docker Container 제작

> DOCKERFILE을 셋팅합니다.

In [85]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: inference


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [86]:
## ECR 등록
## docker login 실행 
registerer.set_aws_ecr(docker=True)

p1:  ['aws', 'ecr', 'get-login-password', '--region', 'ap-northeast-2']
[INFO] target AWS ECR url: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com
p2:  ['sudo', 'buildah', 'login', '--username', 'AWS', '--password-stdin', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com']
[INFO] AWS ECR | docker login result: 
 Login Succeeded!

[INFO] Target AWS ECR repository: 
ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:042969618971:repository/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643",
        "registryId": "042969618971",
        "repositoryName": "ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643",
        "repositoryUri": "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-2312

> Build docker

In [87]:
# docker build 
registerer.build_docker()

STEP 1/12: FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
STEP 2/12: RUN apt-get update
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8062 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [260 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [17.7 kB]
Fetched 8548 kB in 1s (8026 kB/s)
Reading package lists...
STEP 3/12: RUN apt-get install -y apt-utils
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  apt-utils
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 439 kB of archives.
After this operation, 1192 kB of additional disk space will be used.
Get:1 http://deb.debian.org

debconf: delaying package configuration, since apt-utils is not installed


Fetched 439 kB in 0s (11.0 MB/s)
Selecting previously unselected package apt-utils.
(Reading database ... 7034 files and directories currently installed.)
Preparing to unpack .../apt-utils_2.2.4_amd64.deb ...
Unpacking apt-utils (2.2.4) ...
Setting up apt-utils (2.2.4) ...
STEP 4/12: RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20210119).
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu bzip2 cpp cpp-10 dpkg-dev
  g++ g++-10 gcc-10 git-man libasan6 libatomic1 libbinutils libbrotli1
  libc-dev-bin libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0 libctf0
  libcurl3-gnutls libdpkg-perl liberror-perl libgcc-10-dev libgdbm-compat4
  libgomp1 libisl23 libitm1 libldap-2.4-2 liblsan0 libmp

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.32.1 /usr/local/share/perl/5.32.1 /usr/lib/x86_64-linux-gnu/perl5/5.32 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.32 /usr/share/perl/5.32 /usr/local/lib/site_perl) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7, <> line 56.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Selecting previously unselected package libgdbm-compat4:amd64.
Preparing to unpack .../01-libgdbm-compat4_1.19-2_amd64.deb ...
Unpacking libgdbm-compat4:amd64 (1.19-2) ...
Selecting previously unselected package libperl5.32:amd64.
Preparing to unpack .../02-libperl5.32_5.32.1-4+deb11u2_amd64.deb ...
Unpacking libperl5.32:amd64 (5.32.1-4+deb11u2) ...
Selecting previously unselected package perl.
Preparing to unpack .../03-perl_5.32.1-4+deb11u2_amd64.deb ...
Unpacking perl (5.32.1-4+deb11u2) ...
Selecting previously unselected package bzip2.
Preparing to unpack .../04-bzip2_1.0.8-4_amd64.deb ...
Unpacking bzip2 (1.0.8-4) ...
Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../05-libpsl5_0.21.0-1.2_amd64.deb ...
Unpacking libpsl5:amd64 (0.21.0-1.2) ...
Selecting previously unselected package wget.
Preparing to unpack .../06-wget_1.21-1+deb11u1_amd64.deb ...
Unpacking wget (1.21-1+deb11u1) ...
Selecting previously unselected package xz-utils.
Preparing to unpack 

KeyboardInterrupt: 

Selecting previously unselected package libisl23:amd64.
Preparing to unpack .../20-libisl23_0.23-1_amd64.deb ...
Unpacking libisl23:amd64 (0.23-1) ...
Selecting previously unselected package libmpfr6:amd64.
Preparing to unpack .../21-libmpfr6_4.1.0-3_amd64.deb ...
Unpacking libmpfr6:amd64 (4.1.0-3) ...
Selecting previously unselected package libmpc3:amd64.
Preparing to unpack .../22-libmpc3_1.2.0-1_amd64.deb ...
Unpacking libmpc3:amd64 (1.2.0-1) ...
Selecting previously unselected package cpp-10.
Preparing to unpack .../23-cpp-10_10.2.1-6_amd64.deb ...
Unpacking cpp-10 (10.2.1-6) ...
Selecting previously unselected package cpp.
Preparing to unpack .../24-cpp_4%3a10.2.1-1_amd64.deb ...
Unpacking cpp (4:10.2.1-1) ...
Selecting previously unselected package libcc1-0:amd64.
Preparing to unpack .../25-libcc1-0_10.2.1-6_amd64.deb ...
Unpacking libcc1-0:amd64 (10.2.1-6) ...
Selecting previously unselected package libgomp1:amd64.
Preparing to unpack .../26-libgomp1_10.2.1-6_amd64.deb ...
Unpac

> AWS ECR에 docker 이미지를 push 합니다. 

In [38]:
registerer.docker_push()

The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/woosung-test-1207-1526/inference/woosung-test-1207-1526]
ecbaaec88a27: Preparing
069eae61a1fc: Preparing
1f0af2832374: Preparing
07a72215512c: Preparing
2f33dd8c934e: Preparing
6b3d0b913afc: Preparing
277bd6c0df20: Preparing
12d601beac88: Preparing
1b6fd3ad4ce6: Preparing
12d601beac88: Waiting
277bd6c0df20: Waiting
6b3d0b913afc: Waiting
1b6fd3ad4ce6: Waiting
ecbaaec88a27: Pushed
1f0af2832374: Pushed
6b3d0b913afc: Pushed
2f33dd8c934e: Pushed
07a72215512c: Pushed
12d601beac88: Pushed
277bd6c0df20: Pushed
1b6fd3ad4ce6: Pushed
069eae61a1fc: Pushed
latest: digest: sha256:c61adb55d147a9fa4d2ba1b82aaac8413c3e4c5921b41f4fcfb6587971e31bf4 size: 2215
Removing login credentials for https://index.docker.io/v1/


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [88]:
registerer.set_container_uri() # uri도 그냥 입력되게 수정

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643
STEP 11/12: WORKDIR /framework
STEP 12/12: CMD ["python", "main.py"]
COMMIT 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643:latest


Getting image source signatures
Copying blob sha256:1b6fd3ad4ce602924fffb84437331a255e2a9463531a1bd92a15e9e3c4d11523
Copying blob sha256:12d601beac88c4f6011b0d711827f0a08383e99e300eebb128ee17ccbb935e10
Copying blob sha256:277bd6c0df20bd4b73a9dcb29dfbe5019209893d5a8078f3189fc801384555fe
Copying blob sha256:6b3d0b913afc500b93afb9ffbff3c144b25bc17d5780bccd3979ecb29ee1e472
Copying blob sha256:2f33dd8c934e3dbab052f70b9a53cd46dbc810a9fce8cd68609bff634d8a2f00
Copying blob sha256:b487006e190fafb4a6812c2ca875fd7bf6dc509b89d0e73b90cfa64c4ee35877
Copying config sha256:2d5379bb85483079f5b5b79fc8fbe68b54846eb369641cc7475f553b17c4b847
Writing manifest to image destination
Storing signatures


--> 2d5379bb854
Successfully tagged 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643:latest
2d5379bb85483079f5b5b79fc8fbe68b54846eb369641cc7475f553b17c4b847


----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다.

In [89]:
registerer.set_candidate_parameters()
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << candidate_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/inference/artifacts/


> solution_metadata.yaml에 model 경로를 넣어줍니다. <u>(추론 시에만 필요합니다.)</u>

In [90]:
registerer.set_model_uri() # 주의: model은 train artifacts 경로에 존재

[INFO] Completes setting << model_uri >> in solution_metadata.yaml: 
s3://s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/artifacts/


----

#### **STEP-10**. Inference 용 Resource 선택

> 컴퓨팅 자원을 선택합니다.

In [91]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

inference_resource = 'standard'

#----------------------------------------#

registerer.set_resource(inference_resource)


[inference] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-11**. Description 추가

> solution_metadata.yaml에 description을 추가합니다.

In [92]:
# [임시] 수정필요 
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'overview': "AI Advisor Test",

    'input_data': "Test input s3 bucket",
    
    'output_data': "Test output s3 bucket",

    'user_parameters': "Test params",
    
    'algorithm': "ALO"
}
#----------------------------------------#

registerer.set_description(user_description)


<< solution_metadata.yaml >> updated. 
- solution metadata description:
 {'icon': 's3://s3-an2-cism-dev-magna/icons/magna-231208-1643/icon.png', 'title': 'magna-231208-1643', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-cism-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-cism-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}


----

#### **STEP-12**. AI Solution & Instance 등록 !!

> AI Solution 등록을 진행합니다.

In [93]:
# wrangler 정보 등록 
# [임시] 변경 예정
registerer.set_wrangler()

In [94]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

# Edge관련 정보 등록
edgeconductor_interface = {
            'support_labeling': True,
            
            'inference_result_datatype': 'image', # 'image'
            
            'train_datatype': 'table' # 'image'
        }

#----------------------------------------#

registerer.set_edge(edgeconductor_interface)

In [95]:
registerer.register_solution()

[INFO] AI solution register response: 
 {'id': '69e907ae-74bc-4e48-ab04-536aac45caa1', 'name': 'magna-231208-1643', 'scope_ws': 'magna-ws', 'version': {'id': '16b56841-63fd-4faa-905a-64a65fda5f70', 'version': 1, 'train_container_uri': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/train/magna-231208-1643', 'train_container_tag': 'latest', 'inference_container_uri': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-231208-1643/inference/magna-231208-1643', 'inference_container_tag': 'latest', 'metadata_dict': {'version': 1, 'name': 'magna-231208-1643', 'description': {'icon': 's3://s3-an2-cism-dev-magna/icons/magna-231208-1643/icon.png', 'title': 'magna-231208-1643', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-cism-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-cism-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}, 'pipeline

> Solution instance 등록

In [96]:
registerer.register_solution_instance()


[INFO] AI solution interface information: 
 {'name': 'magna-231208-1643', 'solution_version_id': '16b56841-63fd-4faa-905a-64a65fda5f70', 'workspace_name': 'magna-ws'}

[INFO] AI solution instance register response: 
 {'id': 'f0ee86f8-f6f9-47dd-b9d4-502113100835', 'name': 'magna-231208-1643', 'workspace_id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'workspace_name': 'magna-ws', 'solution_version_id': '16b56841-63fd-4faa-905a-64a65fda5f70', 'solution_version': 1, 'metadata_dict': {'version': 1, 'name': 'magna-231208-1643', 'description': {'icon': 's3://s3-an2-cism-dev-magna/icons/magna-231208-1643/icon.png', 'title': 'magna-231208-1643', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-cism-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-cism-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}, 'pipeline': [{'type': 'train', 'dataset_uri': ['s3://s3-an2-cism-dev-magna/ai-solutions/magna-231208-1643/v1/train/data/'], 'artifact_uri': 's3://s3-an

----

> <u>[임시]</u> 추후 Edge Conductor UI에서 진행 예정 

> Stream 등록 

In [48]:
registerer.register_stream()

[INFO] AI solution stream register response: 
 {'id': '0b3cbf5a-31a1-4c88-bf04-6f59af0b0158', 'name': 'woosung-test-1207-1526', 'instance_id': 'e2a76271-926c-485d-9fcf-5b07b0644d07', 'train_dataset_uri': 's3://s3-an2-hyunsoo-dev-magna/instances/woosung-test-1207-1526/woosung-test-1207-1526/train/data/', 'workspace_name': 'magna-ws', 'is_deleted': 0, 'created_at': '2023-12-07 06:29:22'}


> stream run 요청 (train, inference)

In [49]:
registerer.request_run_stream()

[INFO] Run pipeline << inference >> response: 
 {'id': '852525c5-d91c-408b-b86f-c8fb2e08b968', 'name': 'woosung-test-1207-1526-20231207062931', 'workspace_name': 'magna-ws', 'stream_id': '0b3cbf5a-31a1-4c88-bf04-6f59af0b0158', 'train_pipeline_id': 'cc019126-e338-4ea7-929c-9bec43316cb7', 'status': 'running', 'train_dataset_uri': ['s3://s3-an2-hyunsoo-dev-cism/ai-solutions/woosung-test-1207-1526/v1/train/data/'], 'train_artifact_uri': 's3://s3-an2-hyunsoo-dev-magna/instances/woosung-test-1207-1526/0b3cbf5a-31a1-4c88-bf04-6f59af0b0158/train/artifacts/2023/12/07/062931.578/', 'model_version': '0', 'is_deleted': 0, 'created_at': '2023-12-07 06:29:31', 'updated_at': '2023-12-07 06:29:31'}


> pipeline status 조회 

In [51]:
registerer.get_stream_status()

Stream status: Succeeded


> 학습 완료된 artifacts download 

In [ ]:
registerer.download_artifacts()

> download 받은 학습 artifacts로 로컬 환경에서 추론해보기

In [ ]:
# 가상환경이 잘 connected 돼있는지 확인 
!which python 

In [ ]:
# 현재 작업경로 확인 
!pwd

In [ ]:
# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference